# BOTPT Data detide for Central Caldera Station F; Chadwick Method

We detide seafloor pressure measurements taken at The Axial Seamount's Central Caldera (RS03CCAL)  

In [ ]:
%reset -f

In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

#### For mad help with Pandas https://chrisalbon.com
#### For all sorts of great repos https://github.com/rabernat/research_computing.git

#### Read Tide Data F

In [ ]:
tide_file = '/home/jovyan/data/botpt/tidepredictions/pred_F.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
#df_tides_E['height'] = df_tides_E['height'].to_pandas().resample('T').mean()

# columnsTitles=["datetime","height"]
# df_tides_E=df_tides_E.reindex(columns=columnsTitles)
df_tides.head()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]

In [ ]:
df_tides['epoch'] = epoch
df_tides.head()

#### Read Downsampled BOTPT Data

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()
df_botpt.head()

In [ ]:
df_botpt.tail()

#### Combine BOTPT with Tide Predictions

In [ ]:
df_subset = df_tides.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']
df_botpt['tides'] = df_subset.height 
df_botpt.head()

In [ ]:
df_botpt['dtide']=  df_botpt['depth'] - df_botpt['tides']
df_botpt.head()

In [ ]:
#time = list(df_botpt.index.values)
height = list(df_botpt['dtide'].values)

time_int = []
time = list(pd.to_datetime(df_botpt.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2015, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2018, 5, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()